In [5]:
import torch
import torchvision
from torchvision import transforms as T
from torch.utils.data import Dataset, DataLoader, random_split
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
from torchvision.datasets import ImageFolder

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from math import floor, ceil
from sklearn.model_selection import train_test_split

import shutil   
import requests
import functools
import pathlib
from pathlib import Path
import shutil
from tqdm.notebook import tqdm
import os
from collections import defaultdict

from IPython.display import clear_output


matplotlib.rcParams['figure.figsize'] = (20, 5)

In [13]:
url = "https://fall.cv-gml.ru/task_file/15/run_script"
file = "run.py"

cookies = {
    'session': '.eJwlzjFqBDEMheG7uE5hyRpL2ssMkiWREEhgZrcKuXsMaV7xF4_vp5115f3eHs_rlW_t_Ij2aEAqc4g5kpoYRXiVTZjkHQ8HSQCNKNY8AA6ChZCdhkvmjOo6iWzWRNeOVkBRAxAVBhKWwjJnFkQZonsnrN55qSIMc_dqG_K68_rXINMO677qfH5_5tdOWyQMuiEpixlGZzXwfXIQZfTkERAz2u8fxcI-Xg.Y1z7MA.IlLtKJ5SwECajQaH8UOrgYzg1GU',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'ru,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    # 'Cookie': 'session=.eJwlzjFqBDEMheG7uE5hyRpL2ssMkiWREEhgZrcKuXsMaV7xF4_vp5115f3eHs_rlW_t_Ij2aEAqc4g5kpoYRXiVTZjkHQ8HSQCNKNY8AA6ChZCdhkvmjOo6iWzWRNeOVkBRAxAVBhKWwjJnFkQZonsnrN55qSIMc_dqG_K68_rXINMO677qfH5_5tdOWyQMuiEpixlGZzXwfXIQZfTkERAz2u8fxcI-Xg.Y1z7MA.IlLtKJ5SwECajQaH8UOrgYzg1GU',
    'Referer': 'https://fall.cv-gml.ru/login',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.167 YaBrowser/22.7.5.933 Yowser/2.5 Safari/537.36',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Yandex";v="22"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Linux"',
}


In [14]:
def download(url, filename, **kwargs):    
    r = requests.get(url, **kwargs, stream=True, allow_redirects=True)
    if r.status_code != 200:
        r.raise_for_status()  # Will only raise for 4xx codes, so...
        raise RuntimeError(f"Request to {url} returned status code {r.status_code}")
    file_size = int(r.headers.get('Content-Length', 0))

    path = pathlib.Path(filename).expanduser().resolve()
    path.parent.mkdir(parents=True, exist_ok=True)

    desc = "(Unknown total file size)" if file_size == 0 else ""
    r.raw.read = functools.partial(r.raw.read, decode_content=True)  # Decompress if needed
    with tqdm.wrapattr(r.raw, "read", total=file_size, desc=desc) as r_raw:
        with path.open("wb") as f:
            shutil.copyfileobj(r_raw, f)

    return path

In [15]:
download(url, file, cookies=cookies, headers=headers)

  0%|          | 0/5422 [00:00<?, ?it/s]

PosixPath('/home/alexkkir/cv-hw10-road-signs/run.py')

In [11]:
shutil.unpack_archive('data.zip', 'template')

In [12]:
os.remove('data.zip')

In [11]:
shutil.move('additional_files', '/mnt/ssd1/datasets/alexkkir/road_signs/additional_files')

'/mnt/ssd1/datasets/alexkkir/road_signs/additional_files'

In [ ]:
! ln -s /mnt/ssd1/datasets/alexkkir/road_signs/additional_files additional_files